In [1]:
import pandas as pd
import numpy as np

In [2]:
d = pd.read_csv('ph.csv', compression='gzip')

/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
d.head()

,id,creation_date,post_id,post_history_type_id,revision_guid,user_id,text,comment
0,28369829,2012-08-24 08:18:33.29 UTC,12105586,1,95c45c9a-aff1-4b7c-adc2-76c727c6a930,1165423.0,Loading String Resources from A Different Path,NaN
1,95514764,2015-07-20 18:28:11.287 UTC,31523754,1,c8ef2715-59bb-429b-ad39-25086ec4e123,5136201.0,System.Data.SqlClient.SqlException: Incorrect ...,NaN
2,187966461,2018-12-17 09:16:42.123 UTC,53812067,1,72e5ae09-4edc-44f2-ae53-8eacda39a9f4,2105218.0,Do multiple jdbc input run simultaneously or s...,NaN
3,8155373,2010-10-07 21:33:26.907 UTC,3886039,1,f419429b-9756-43ff-baf4-5927f589c73c,56621.0,"""Test with debugger"" in ReSharper?",NaN
4,40793970,2013-05-17 16:18:08.577 UTC,16613724,1,ef698524-da6f-4a1f-b0ba-310b38e730d7,2394636.0,jQuerymobile: Close a popup when opening a panel,NaN


In [71]:
# demo from online:
import string
import math

tokenize = lambda doc: doc.lower().split(" ")

document_0 = "China has a strong economy that is growing at a rapid pace. However politically it differs greatly from the US Economy."
document_1 = "At last, China seems serious about confronting an endemic problem: domestic violence and corruption."
document_2 = "Japan's prime minister, Shinzo Abe, is working towards healing the economic turmoil in his own country for his view on the future of his people."
document_3 = "Vladimir Putin is working hard to fix the economy in Russia as the Ruble has tumbled."
document_4 = "What's the future of Abenomics? We asked Shinzo Abe for his views"
document_5 = "Obama has eased sanctions on Cuba while accelerating those against the Russian Economy, even as the Ruble's value falls almost daily."
document_6 = "Vladimir Putin was found to be riding a horse, again, without a shirt on while hunting deer. Vladimir Putin always seems so serious about things - even riding horses."

all_documents = [document_0, document_1, document_2, document_3, document_4, document_5, document_6]

tokenized_documents = [tokenize(d) for d in all_documents] # tokenized docs
all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
print(tokenized_documents)

[['china', 'has', 'a', 'strong', 'economy', 'that', 'is', 'growing', 'at', 'a', 'rapid', 'pace.', 'however', 'politically', 'it', 'differs', 'greatly', 'from', 'the', 'us', 'economy.'], ['at', 'last,', 'china', 'seems', 'serious', 'about', 'confronting', 'an', 'endemic', 'problem:', 'domestic', 'violence', 'and', 'corruption.'], ["japan's", 'prime', 'minister,', 'shinzo', 'abe,', 'is', 'working', 'towards', 'healing', 'the', 'economic', 'turmoil', 'in', 'his', 'own', 'country', 'for', 'his', 'view', 'on', 'the', 'future', 'of', 'his', 'people.'], ['vladimir', 'putin', 'is', 'working', 'hard', 'to', 'fix', 'the', 'economy', 'in', 'russia', 'as', 'the', 'ruble', 'has', 'tumbled.'], ["what's", 'the', 'future', 'of', 'abenomics?', 'we', 'asked', 'shinzo', 'abe', 'for', 'his', 'views'], ['obama', 'has', 'eased', 'sanctions', 'on', 'cuba', 'while', 'accelerating', 'those', 'against', 'the', 'russian', 'economy,', 'even', 'as', 'the', "ruble's", 'value', 'falls', 'almost', 'daily.'], ['vladim

In [72]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [73]:
%timeit a = jaccard_similarity(tokenized_documents[2],tokenized_documents[4])

4.22 µs ± 89.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [74]:
# term-frequency inverse document frequency
def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

def sublinear_term_frequency(term, tokenized_document):
#     print(tokenized_document.count(term))
#     print(tokenized_document, term)
    try:
        return 1 + math.log(tokenized_document.count(term))
    except ValueError:
        return 0
    

def augmented_term_frequency(term, tokenized_document):
    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))

def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

In [75]:
idf_values = inverse_document_frequencies(tokenized_documents)
print(idf_values['abenomics?'])
# 2.94591014906
print(idf_values['the'])

2.9459101490553135
1.336472236621213


In [76]:
def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

In [77]:
%time tfidf_representation = tfidf(all_documents)
# print(tfidf_representation[0], document_0)
# doc vector and document

CPU times: user 1.55 ms, sys: 0 ns, total: 1.55 ms
Wall time: 1.56 ms


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)

sklearn_representation = sklearn_tfidf.fit_transform(all_documents)
print(tfidf_representation[0])
print(sklearn_representation.toarray()[0].tolist())
print(document_0)

[2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 1.336472236621213, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 3.8142592685777856, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 2.9459101490553135, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8472978603872037, 2.9459101490553135, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 1.8472978603872037, 0.0, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.31019096605521496, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18320378146489946, 0.0, 0.18320378146489946, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23957330918096045, 0.0, 0.0, 0.0, 0.183203

In [49]:
d['text'][1]

"System.Data.SqlClient.SqlException: Incorrect syntax near the keyword 'FROM'"

In [7]:
import pandas as pd
d2 = pd.read_csv('posts_questions.csv', compression='gzip')

/home/neal/miniconda3/envs/keras/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
def clean(line):
#     print(type(line))
    line = line.lower().replace("\n"," ").replace("\r","").replace(',',"").replace(">","> ")
    return line
# clean(d2['body'][0])
# d2.to_csv('pq2.csv', sep='~')
# col = []
# for text in d2['body']:
#     col.append(clean(text))
# d2['cleaned'] = col
# print(d2['cleaned'][0])
# d2['cleaned'].to_csv('pq_clean.csv')
import pandas as pd
d2 = pd.read_csv("pq_clean.csv", names=['index','body'])
# print(d2['body'][0])

In [1]:
# d.head()
# d = 5
def word_count(row):
    return len(row.split(" "))
#     print(row)
#     l = row.split(',')[2]
#     print(l)
#     return len(l.split(" "))

from pyspark import SparkContext
def process(filename):
    sc = SparkContext(appName="Text Process").getOrCreate()
    data = sc.textFile(filename)
#     sc.addPyFile("mfun.pyx")
    scores = sc.parallelize(data).map(mfun.word_count).collect()
    sc.stop()
    return scores

In [5]:
import random
print(random.randint(0,1000))

285


In [1]:
# print(d2[:10])
# sc.stop()
# d3 = d2[:100]
# d3.to_csv('pq.csv')
# from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkSession.builder.appName("TextProcess").getOrCreate()
# sc = SparkContext(appName = 'TextProcess').getOrCreate()
# def line(a):
#     sp = a.split(",")
#     return a.split("~")
# process('pq.csv')
def word_count(row):
    return len(row.split(" "))

# files = sc.textFile("pq_clean.csv") \
# .map(lambda line: line.split(",")[1]) \
# .cache()
files = sc.read.format("csv").option("header","false").load("pq_clean.csv")
f = files.take(10)

import random
# sample = files.take(random.randint(1,1000))
# print(sample)
# .map(word_count) \
# .cache()
# .sum()
#     .map(lambda line: line.split(",")) \
#     .collect()

In [2]:
# print(f[0])
# print(files.columns)

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


tk = Tokenizer(inputCol="_c1", outputCol="words")
words = tk.transform(files)
# w = words.collect()
hashingTF = HashingTF(inputCol="words", outputCol='rawFeatures')
tf = hashingTF.transform(words)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(tf)
rescaledData = idfModel.transform(tf)
rescaledData.cache()

# tf = hashingTF.transform(words)
# tf.cache()
rescaledData.select("_c0", "features").show() 

+---+--------------------+
|_c0|            features|
+---+--------------------+
|  0|(262144,[721,2711...|
|  1|(262144,[3727,392...|
|  2|(262144,[170,4146...|
|  3|(262144,[1487,179...|
|  4|(262144,[1536,276...|
|  5|(262144,[2565,822...|
|  6|(262144,[2978,303...|
|  7|(262144,[602,2835...|
|  8|(262144,[438,489,...|
|  9|(262144,[1536,161...|
| 10|(262144,[54,8926,...|
| 11|(262144,[1538,735...|
| 12|(262144,[966,3067...|
| 13|(262144,[1681,248...|
| 14|(262144,[1536,222...|
| 15|(262144,[9639,244...|
| 16|(262144,[353,9639...|
| 17|(262144,[170,9615...|
| 18|(262144,[6886,142...|
| 19|(262144,[9193,963...|
+---+--------------------+
only showing top 20 rows



In [4]:
sample = sc.createDataFrame([(f[2][0], f[2][1]),],['_c0','_c1'])
# print(sample)
s_tk = tk.transform(sample)
candidateTf = hashingTF.transform(s_tk)
candidateTfIdf = idfModel.transform(candidateTf)
candidateTfIdf.select("_c0", "features").show() 
candidate = candidateTfIdf.select("features")
# candidate.
# print(candidate.toList)
# similarities = rescaledData.select(norm("features")).rdd.map(lambda v: v[0])#/(v[0].norm(2)*candidate[0].norm(2)))
# s = similarities.collect()
# print(similarities)
# similarities = rescaledData.select("features").(lambda v: v.norm(2))#/(v.norm(2)*candidate.norm(2)))

# similarities = rescaledData.select("features").map(lambda v: v.dot(candidateTfIdf.select('features')) / (v.norm(2) * candidateTfIdf.select('features').norm(2)))
# %time topFive = sorted(enumerate(similarities.collect()), key= lambda kv: kv[1])[0:5]
# for idx, val in topFive:
#     print("doc '%s' has score %.4f" % (idx, val))

+---+--------------------+
|_c0|            features|
+---+--------------------+
|  2|(262144,[170,4146...|
+---+--------------------+



In [ ]:
from pyspark.ml.feature import Normalizer 
normalizer = Normalizer(inputCol="features", outputCol="norm") 
data = normalizer.transform(rescaledData)

from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix 
mat = IndexedRowMatrix(data.select("_c0", "norm").rdd.map(lambda row: IndexedRow(row._c0, row.norm.toArray()))).toBlockMatrix() 
dot = mat.multiply(mat.transpose())
dot.toLocalMatrix().toArray()

In [ ]:
## alternatively, 
# dot_udf = psf.udf(lambda x,y: float(x.dot(y)), DoubleType()) data.alias("i").join(data.alias("j"), psf.col("i.ID") < psf.col("j.ID"))\ 
# .select( psf.col("i.ID").alias("i"), psf.col("j.ID").alias("j"), dot_udf("i.norm", "j.norm").alias("dot"))\ .sort("i", "j")\ 
# .show()

In [31]:
# from pyspark.sql.functions import udf
# from pyspark.sql.types import array
# cosine_similarity = udf(lambda x: x.norm(2))
# similarities = rescaledData.withColumn('similarity', cosine_similarity(rescaledData['features']))
# similarities.select('similarity').show()

Py4JJavaError: An error occurred while calling o1464.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 91, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:90)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:89)
	at scala.collection.Iterator$$anon$12.nextCur(Iter

In [7]:
# tf.take(1)
# %time idf = IDF(inputCol="rawfeatures", outputCol="features").fit(tf)
# %time tfidf = idf.transform(tf)
# %time idf = IDF().fit(files)
# %time tfidf = idf.transform(files)

CPU times: user 12.5 ms, sys: 5.27 ms, total: 17.7 ms
Wall time: 41.4 s
CPU times: user 1.55 ms, sys: 4.86 ms, total: 6.41 ms
Wall time: 27.9 ms


In [8]:
# a = tfidf.collect()
# print(a[2])
# tfidf.

In [17]:
# type(f)

list

In [29]:
# print(a[1])
# look at one file
# print(f[2])
sample = sc.createDataFrame([(f[2][0], f[2][1]),],['_c0','_c1'])
# print(sample)
s_tk = tk.transform(sample)
candidateTf = hashingTF.transform(s_tk)
candidateTfIdf = idfModel.transform(candidateTf)
candidateTfIdf.select("_c0", "features").show() 
candidate = candidateTfIdf.select("features")
print(candidate)
similarities = rescaledData.select("features").rdd.map(lambda v: v.norm(2))#/(v.norm(2)*candidate.norm(2)))

# similarities = rescaledData.select("features").map(lambda v: v.dot(candidateTfIdf.select('features')) / (v.norm(2) * candidateTfIdf.select('features').norm(2)))
%time topFive = sorted(enumerate(similarities.collect()), key= lambda kv: kv[1])[0:5]
for idx, val in topFive:
    print("doc '%s' has score %.4f" % (idx, val))
# c = candidateTfIdf.collect()
# print(c[0]['features'], )

# sample = s_tk.collect()
# print(sample)



+---+--------------------+
|_c0|            features|
+---+--------------------+
|  2|(262144,[170,4146...|
+---+--------------------+

DataFrame[features: vector]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 48.0 failed 1 times, most recent failure: Lost task 2.0 in stage 48.0 (TID 110, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 1527, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'norm' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
   

NameError: name 'topFive' is not defined

In [ ]:
# compute similarity


In [30]:
print(candidateTfIdf)
print(a[2])

[Row(_c0='2', _c1='"<p> i am making a class to store a reference to an object if given an lvalue in the constructor and to store a copy if given an rvalue. this is fine and compiles:</p>   <pre> <code> template &lt;typename obj_t&gt; struct holder {     obj_t obj;      template &lt;typename obj_ref_t&gt;     holder(obj_ref_t &amp;&amp; o)         : obj(std::forward&lt;obj_ref_t&gt;(o))     {} };  template &lt;typename obj_t&gt; holder&lt;obj_t&gt; make_held(obj_t &amp;&amp; o) {     return holder&lt;obj_t&gt;(std::forward&lt;obj_t&gt;(o)); } </code> </pre>   <p> however when i add a destructor (even an empty one) to the class i get a compiler error:</p>   <pre> <code> invalid instantiation of non-const reference of type ""obj_t &amp;"" from an rvalue of type ""holder&lt;obj_t &amp;&gt;"" </code> </pre>   <p> why does adding a destructor somehow invoke a constructor call to create a wrapped object out of an already wrapped object?</p> "', words=['"<p>', 'i', 'am', 'making', 'a', 'class'

In [10]:
# look at one file
# candidate = clean(d2['body'][4])

sample = [2]
# print(sample)
candidate = sample
# # print(candidate)
candidateTf = hashingTF.transform(candidate)
candidateTfIdf = idf.transform(candidateTf)
similarities = tfidf.map(lambda v: v['features'].dot(candidateTfIdf['features']) / (v['features'].norm(2) * candidateTfIdf['features'].norm(2)))
%time topFive = sorted(enumerate(similarities.collect()), key= lambda kv: kv[1])[0:5]
for idx, val in topFive:
    print("doc '%s' has score %.4f" % (idx, val))

# ct = candidateTfIdf

print(ct)
# sims = tfidf.collect()
# similarities = tfidf.map(lambda v: v.dot(candidateTfIdf) / (v.norm(2) * candidateTfIdf.norm(2)))
# for i in enumerate(similarities.collect()):
#     print(i)

# %time topFive = sorted(enumerate(similarities.collect()), key= lambda kv: kv[1])[0:5]
# # topFive = sorted(enumerate(similarities.collect()), key=lambda (k, v): -v)[0:5]
# for idx, val in topFive:
#     print("doc '%s' has score %.4f" % (d2['body'][idx], val))

AttributeError: 'list' object has no attribute '_jdf'

In [14]:
print(a[7])
print(" ")
print(ct)

(1048576,[7357,7734,24138,26255,27527,35920,40200,40649,105642,173606,186435,211440,214770,238153,252894,265159,274911,279832,292370,295235,296409,335453,372567,388504,410631,425102,437751,469732,485029,493959,517893,518279,540294,584844,681187,702216,702740,734443,777769,785264,840959,875351,877522,891534,897367,945367,948889,968035],[2.5484077490414774,2.5435678559399135,12.466094245807447,0.2689800908220133,3.1341925734651905,0.1863456276389978,0.01734205537160227,8.387283702838273,0.28634343156558556,0.02983216643475024,0.3642313653768495,0.01121049568999458,1.469856219228363,0.004882414463569328,0.0,0.002552833500591609,2.6128832698841125,0.2076907573027907,2.5780233673305317,1.0552661200084725,0.0011603603499318554,0.06073191408947854,0.20835756772279174,0.029839963284632263,2.5206360031491606,7.840895392940535,0.21155021886480344,0.12867210995166206,7.057487180265828,0.0,2.8360319818018267,3.8923935691667637,1.4331880872079645,8.992740537910254e-05,1.3219377634221123,0.003898901

In [7]:
# sims[5].dot(sims[8])
# print(sims[8].norm(2))
# print(sims[1])
# sims[1].dot(sims[2])
# print(ct)
# print(sims[1])
tf_store = tf.take(100)
transformed = idf.transform(tf_store).collect()

MemoryError: 

In [ ]:
# tf_store[4]
sims[1].dot(transformed[1])

In [ ]:
# from pyspark.sql.types import *
ct2 = ct.toArray()
sims2 = []
for i in sims:
    sims2.append(i.toArray())
sims2 = np.array(sims2)
# print(ct2.shape)
# print(ct2.max())
# print(sims[0])

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47030)
Traceback (most recent call last):
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func()

In [36]:
a = similarities.collect()
# print(min(a))

0.0


In [18]:
# print(max(a))
# print(candidateTf.shape)
print(hashingTF.numFeatures)

1048576


In [19]:
# df = pd.read_csv("pq_clean.csv")
# d2.head()

,index,body
0,0,<p> i am trying to create a report to display ...
1,1,<p> i understand payments are a tricky thing b...
2,2,<p> i am making a class to store a reference t...
3,3,<p> i am trying to compare performance of <cod...
4,4,"<p> when i am reading the code of <a href=""htt..."


In [78]:
%time tfidf_representation = tfidf(df)

CPU times: user 4.29 ms, sys: 12 µs, total: 4.31 ms
Wall time: 4.22 ms


In [90]:
# print(tfidf_representation[1])

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
# tokenize = lambda doc: doc.lower().split(" ")
# sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True)#, tokenizer=tokenize)
# %time sklearn_representation = sklearn_tfidf.fit_transform(d2['body'])

In [51]:
hashidf = HashingVectorizer()
skh = hashidf.fit_transform(d2['body'])


In [52]:
print(skh.shape)

(344722, 1048576)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37560)
Traceback (most recent call last):
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func()

In [21]:
# d2.head()
# print(sklearn_tfidf.get_stop_words())
# %time sklearn_tfidf.transform(d2['body'][:1])
import numpy as np
print(sklearn_representation.shape)
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude = np.linalg.norm(vector1)*np.linalg.norm(vector2)
    if not magnitude:
        return 0
    return dot_product/magnitude

(344722, 1517363)


In [80]:
# print(np.sqrt(np.sum(candidate**2)))
print(np.linalg.norm(candidate))

1.0000000000000002
1.0000000000000002


In [30]:
candidate = sklearn_tfidf.transform(d2['body'][:1]).toarray()[0]
target = sklearn_representation[3].toarray()
print(candidate[0].shape, target[0].shape)
# print()
sim = []

# a = np.dot(candidate[0], target[0])
# %time a = cosine_similarity(candidate[0], target[0])
# print(a)
# print(a)

# print(candidate.shape, sklearn_representation[3].shape)
a = cosine_similarity(candidate, sklearn_representation[2].toarray()[0])
# print(a)
# print(a)
# %time np.dot(candidate.T, sklearn_representation[2])
# for count, i in enumerate(sklearn_representation[:1000]):
#     sim.append([cosine_similarity(candidate, i.toarray()[0]), count])
# # print(sim)
# print(sorted(sim)[:5])

def compare(candidate, n):
    sim = []
    for count, i in enumerate(sklearn_representation[:n]):
        sim.append([cosine_similarity(candidate, i.toarray()[0]), count])
    return sorted(sim, reverse=True)[:5]
# print(len(df))
# print(sklearn_representation[1])

() (1517363,)


In [31]:
%time compare(candidate, 10000) # 1.5s for 100, 12.8s for 1000

CPU times: user 1min 26s, sys: 1.56 s, total: 1min 27s
Wall time: 44 s


[[1.0000000000000002, 0],
 [0.15263288080462353, 2233],
 [0.13922132245710933, 2108],
 [0.11908126026567464, 4772],
 [0.11846256923066482, 2416]]

In [35]:
# print(d2['body'][2233])
# print(" ")
# print(d2['body'][0])

<p> i am new in using sql with delphi. i have a table i.e.:</p>   <pre> <code> columnname1 columnname2 columnname3 value 1a    value2a     value3a value 1b    value2b     value3b value 1c    value2c     value3c </code> </pre>   <p> and so on. i have the table displayed on a <code> dbgrid</code>  so currently the adodataset is all connected ok. however i have a small loop</p>   <pre> <code> for i := 0 to adodataset.fieldcount - 1 do begin   writeln(wolfileadodataset.fields[i].asstring); end; </code> </pre>   <p> that writes to a text file. this text file does not have the column names and lists the fields one per line.</p>   <pre> <code> value 1a value 2a value 3a value 1b value 2b value 3b </code> </pre>   <p> etc. how can i get it to list in a row format similar to how i've represented the complete table above?</p> 
 
<p> i am trying to create a report to display a summary of the values of the columns for each row.   a basic analogy would an inventory listing.  say i have about 15 loc

In [59]:
# print(sklearn_representation[0])
# print(candidate)
# for i in candidate:
# import scipy.sparse.csr_matrix
# from scipy.sparse.csr_matrix import todense
a = candidate.toarray()
# print(a)

[[0.06933385 0.         0.         ... 0.         0.         0.        ]]


In [29]:
print(sorted([0,2,3,6,4,8,3]))

[0, 2, 3, 3, 4, 6, 8]
